In [1]:
from arcgis.gis import GIS
import os
from multiprocessing import Process, Queue

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import sys
sys.path.append('../src')

import reach_tools

%load_ext autoreload
%autoreload 2

In [2]:
gis = GIS(username=os.getenv('ARCGIS_USERNAME'), password=os.getenv('ARCGIS_PASSWORD'))
gis

GIS @ https://knu2xs.maps.arcgis.com version:7.1

In [3]:
url_reach_line = os.getenv('URL_REACH_LINE')
url_reach_centroid = os.getenv('URL_REACH_CENTROID')
url_reach_points = os.getenv('URL_REACH_POINT')

lyr_reach_line = reach_tools.ReachFeatureLayer(url_reach_line, gis)
lyr_reach_centroid = reach_tools.ReachFeatureLayer(url_reach_centroid, gis)
lyr_reach_points = reach_tools.ReachPointFeatureLayer(url_reach_points, gis)

In [14]:
df_pts = lyr_reach_points.query().sdf
df_ln = lyr_reach_line.query().sdf
df_cnt = lyr_reach_centroid.query().sdf

In [20]:
def delete_null_geometry_features(lyr):
    df = lyr.query().sdf
    null_oid_arr = df[df.SHAPE.isnull()].OBJECTID.values
    if len(null_oid_arr):
        objectid_lst = ','.join([str(val) for val in null_oid_arr])
        lyr.edit_features(deletes=objectid_lst)
        print(f'deleted {len(null_oid_arr)} null geometry records')
    else:
        print('no null geometries')

In [21]:
for lyr in lyr_reach_line, lyr_reach_centroid, lyr_reach_points:
    delete_null_geometry_features(lyr)

no null geometries
no null geometries
no null geometries
